### Задание по программированию: Линейная регрессия: прогноз оклада по описанию вакансии

In [105]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer

from scipy.sparse import hstack

import re

#### 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.

In [91]:
train_data = pd.read_csv("./salary-train.csv")

In [106]:
train_data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


#### 2. Проведите предобработку:
##### • Приведите тексты к нижнему регистру.

##### • Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.

In [93]:
def text_transform(t):
    return t.apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', x.lower()))

In [107]:
train_data["FullDescription"] = text_transform(train_data["FullDescription"])

##### Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [108]:
tfidf_vectorizer = TfidfVectorizer(min_df = 5)
X_train_description = tfidf_vectorizer.fit_transform(train_data["FullDescription"])

##### Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку ’nan’.

In [109]:
train_data["LocationNormalized"].fillna("nan", inplace = True)
train_data["ContractTime"].fillna("nan", inplace = True)

##### Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [110]:
dict_vectorizer = DictVectorizer()
X_train_categ = dict_vectorizer.fit_transform(train_data[['LocationNormalized', 'ContractTime']].to_dict('records'))

##### Объедините все полученные признаки в одну матрицу "объектыпризнаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [112]:
X_train_prepared = hstack([X_train_description, X_train_categ])

#### 3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.

In [113]:
model = Ridge(alpha = 1)
model.fit(X_train_prepared, train_data["SalaryNormalized"])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

#### 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

In [100]:
test_data = pd.read_csv("./salary-test-mini.csv")

In [101]:
test_data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [102]:
X_test_description = tfidf_vectorizer.transform(text_transform(test_data["FullDescription"]))
X_test_categ = dict_vectorizer.transform(test_data[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_prepared = hstack([X_test_description, X_test_categ])

In [103]:
predicted = model.predict(X_test_prepared)

In [104]:
predicted

array([56560.75731482, 37206.84453909])